# Паспорт проекта "Реализация алгоритма Моргана на языке Python"
Автор: Ярцев Кирилл; yartsevkv@yandex.ru; tg @sbesbesbe
Цель: реализовать алгоритм канонической нумерации атомов в молекуле
Задачи:
    1.Ознакомиться с описанием алгоритма;
    2.Определиться с типом исходных данных;
    3.Написать программу, выполняющую поставленную цель.
Источники:
    1.Введение в хемоинформатику: Компьютерное представление химических структур: учеб. пособие / Т.И. Маджидов, И.И. Баскин, И.С. Антипин, А.А. Варнек. – Казань, Москва, Страсбург, 2020. – 173 с.
    2.Документация NumPy

In [1]:
import numpy as np

In [176]:
def mol_read(fileName):
    text = open(fileName)                                   #открываем MOL-файл
    lines = text.readlines()[3:]                            #читаем информативные строки
    generalData = lines[0].split()                            #разделяем нулевую строку по пробелам
    atomCnt = int(generalData[0])                             #число атомов в молекуле
    bondCnt = int(generalData[1])                             #число связей в молекуле
    bondBlc = lines[atomCnt+1:atomCnt+1+bondCnt]            #вчитываем блок связей из MOL-файла
    
    atomLabels = []                                         #создание массива с обозначениями неводородных атомов
    for line in lines[1:atomCnt+1]:
        if line.split()[3] != 'H':
            atomLabels.append(line.split()[3])
    atomLabels = np.asarray(atomLabels)
    
    atomLabelsH = []                                         #создание массива с обозначениями всех атомов
    for line in lines[1:atomCnt+1]:
        atomLabelsH.append(line.split()[3])
    atomLabelsH = np.asarray(atomLabelsH)
    
    Hatoms = len(atomLabelsH) - len(atomLabels)              #число атомов водорода
    
    connectivityTab = np.zeros((atomCnt-Hatoms,atomCnt-Hatoms),dtype = int)  #матрица связанности неводородных атомов
    
    hydrogenIndexes = []                                         #создание списка с индексами атомов водорода
    for line in lines[1:atomCnt+1]:
        if line.split()[3] == 'H':
            hydrogenIndexes.append(lines[1:atomCnt+1].index(line))
    
    for line in bondBlc:
        i = int(line.split()[0])-1                          #номер первого атома связи в нумерации MOL-файла
        j = int(line.split()[1])-1                          #номер второго атома связи в нумерации MOL-файла
        k = int(line.split()[2])                            #порядок связи
        if(k!=0) and i not in hydrogenIndexes and j not in hydrogenIndexes:
            connectivityTab[i][j] = 1                          #в матрице связанности проставляется связанность
            connectivityTab[j][i] = 1

    return connectivityTab, atomLabels

In [177]:
def morgan_algo(fileName):
    connectivityTab, atomLabels = mol_read(fileName)
    degrees = np.sum(connectivityTab!=0,axis=0) #кол-во соседних атомов у данного
    
    neighboring_indexes = []                                              #создание списка соседей данного атома с исходными индексами
    for i in range(len(atomLabels)):
        neighboring_indexes.append(list(np.where(connectivityTab[i]>0)[0]))
    
    print("Соседние атомы для каждого атома:")                              #вывод соседей атома 
    for i in range(len(atomLabels)):
        print('{} {}'.format(atomLabels[i], neighboring_indexes[i]))
    
    #суммирование extended connectivity
    degreesOLD = degrees
    degreesNEW = []
    numberOfIter = 0
    while (np.unique(degreesNEW)).size > (np.unique(degreesOLD)).size or (np.array(degreesNEW)).size == 0:
        if (np.array(degreesNEW)).size != 0:
            degreesOLD = degreesNEW
        numberOfIter += 1
        neighboring_degrees=[[degreesOLD[j] for j in i] for i in neighboring_indexes]
        degreesNEW = [sum(i) for i in neighboring_degrees]
        print((np.unique(degreesNEW)).size,"различных значений связаннности после итерации",numberOfIter,":",np.unique(degreesNEW))
    
    # список morgan будет хранить моргановскую нумерацю
    morgan = []
    degreesNEW = np.array(degreesNEW)
    
    # начинаем с поиска атома с наибольшим EC, помещаем его в список
    morgan.append(np.where(degreesNEW.argsort()[::-1]==0)[0].tolist())
    
    #дополняем список другими атомами
    while len(morgan)<len(atomLabels):
        for i in morgan:
            neighboring_degrees = [] #список для хранения значений связанностей соседних атомов
            cp_index = [] #список для хранения индексов соседей для того атома, по которому идет цикл
            for j in neighboring_indexes[i[0]]: #записываем соседей данного атома
                if j not in morgan:
                    neighboring_degrees.append(degreesNEW[j])
                    cp_index.append(j)
             #сортируем атомы по EC и проставляем нумерацию в соответствии с ней
            sorted_cp_index =[]
            for index in np.array(neighboring_degrees).argsort()[::-1]:
                sorted_cp_index.append(cp_index[index])

            for index in sorted_cp_index: #добавляем номер в список
                if index not in morgan:
                    morgan.append([index])
        print("Результат канонизации:")

        mol_index =0
        for i in range(len(atomLabels)):
            print('{}{} :{}'.format(atomLabels[i], mol_index,morgan[i]))
            mol_index +=1

In [178]:
molFile  = str(input())  
#iterCount = int(input())  
morgan_algo(molFile)


cholesterol.sdf
Соседние атомы для каждого атома:
C [1, 3, 27]
C [0, 2]
C [1, 25]
C [0, 4]
C [3, 5, 25]
C [4, 6]
C [5, 7]
C [6, 8, 24]
C [7, 9, 20]
C [8, 10]
C [9, 11]
C [10, 12, 20]
C [11, 13, 19]
C [12, 14]
C [13, 15]
C [14, 16]
C [15, 17, 18]
C [16]
C [16]
C [12]
C [8, 11, 21, 22]
C [20]
C [20, 23]
C [22, 24]
C [7, 23, 25]
C [2, 4, 24, 26]
C [25]
O [0]
6 различных значений связаннности после итерации 1 : [3 4 5 6 8 9]
16 различных значений связаннности после итерации 2 : [ 4  5  6  8  9 10 11 13 14 15 17 20 22 23 27 28]
18 различных значений связаннности после итерации 3 : [11 14 16 17 18 21 27 28 29 33 34 36 38 43 53 57 59 65]
25 различных значений связаннности после итерации 4 : [ 16  29  34  36  43  48  54  65  67  76  82  89  90  93  95  99 101 103
 108 132 135 158 166 183 195]
27 различных значений связаннности после итерации 5 : [ 43  66  76  88  91 103 151 169 178 183 195 208 222 225 234 250 253 255
 259 303 354 393 406 448 457 459 460]
27 различных значений связаннности посл